In [246]:
import requests
# import time
from time import sleep, time
import getpass

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import TimeoutException

import pandas as pd
import numpy as np

import asyncio
import warnings
warnings.filterwarnings("ignore")

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

In [247]:
chrome_options = Options()
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-plugins-discovery')
chrome_options.add_argument('--disable-web-security')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--disable-popup-blocking')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--start-maximized')

In [248]:
USERNAME = getpass.getuser()
if USERNAME == "runlanlana":
    PATH = None
else:
    PATH = '/Users/Benjaminzeisberg'
    PATH = 'INSERt'
print(f"{USERNAME}'s PATH: {PATH}")

benjaminzeisberg's PATH: INSERt


In [249]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")

if USERNAME == "runlanlana":
    driver = webdriver.Chrome(options=options)
else:
    driver = webdriver.Chrome(options=options, executable_path=PATH)
driver.set_page_load_timeout(25)

In [250]:
row2k_results_url = "https://www.row2k.com/results/" # first page url
driver.get(row2k_results_url)

In [251]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [252]:
list_year = []
selector_options = soup.findAll('option',attrs={'selected':''})
for years in selector_options[1:]:
    list_year.append(str(years).split()[-1][:4])

In [253]:
print(list_year)

['2022', '2021', '2020', '2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009', '2008', '2007', '2006', '2005', '2004', '2003', '2002', '2001', '2000', '1999', '1998', '1997']


In [254]:
from selenium.webdriver.support.ui import Select

In [255]:
# Find options
# find the dropdown element by its ID
dropdown = driver.find_element(By.XPATH, '//*[@id="full"]/div[1]/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr/td/div[3]/form/select')

# create a Select object
select = Select(dropdown)

# get all options of the dropdown
options = select.options
print(len(options))
# print(options)
# # iterate over the options and select each one


27


In [256]:
def links():
    link_ls = []
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Find the span containing the desired text
    target_span = soup.find_all('span', class_='newscat2', text='EARC HW Men')

    for span in target_span:
        # Find the parent ul element
        ul_element = span.find_next('ul')
        
        if ul_element:
            # Extract all links within the ul element
            links = ul_element.find_all('a')
            
            # print(links)
            # # Print the href attribute of each link
            for link in links:
                # print(link.get('href'))
                link_ls.append(link.get('href'))
        else:
            print("No <ul> found after the target span.")
    return link_ls

# Function to select all years and scrape all links for pages
links_all_years = []
for i in range(0, len(options)):
    dropdown = driver.find_element(By.XPATH, '//*[@id="full"]/div[1]/table/tbody/tr/td/table/tbody/tr[2]/td/table/tbody/tr/td/div[3]/form/select')

    # create a Select object
    select = Select(dropdown)

    # get all options of the dropdown
    options = select.options
    select.select_by_index(i)
    links_list = links()
    links_all_years.append(links_list)

In [257]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")
driver = webdriver.Chrome(options=options, executable_path=PATH)

In [258]:
html_pages = []
for i, items in enumerate(links_all_years):
    print(f'Step {i}-27')
    html_content = []
    for link in items:
        driver.get(f"https://www.row2k.com{link}")
        html_content.append(BeautifulSoup(driver.page_source, 'html.parser'))
    html_pages.append(html_content)

Step 0-27
Step 1-27
Step 2-27
Step 3-27
Step 4-27
Step 5-27
Step 6-27
Step 7-27
Step 8-27
Step 9-27
Step 10-27
Step 11-27
Step 12-27
Step 13-27
Step 14-27
Step 15-27
Step 16-27
Step 17-27
Step 18-27
Step 19-27
Step 20-27
Step 21-27
Step 22-27
Step 23-27
Step 24-27
Step 25-27
Step 26-27


In [259]:
# tds = soup.find_all('td')
def find_idx(ls):
    idx = []
    for i, item in enumerate(ls):
        if str(item).find('colspan') != -1:
            idx.append(i)
        else:
            continue
    return idx

def results(soup):
    '''
    Find Results of page, given soup element of that page and create List of columns [boat, team, time]
    '''
    tds = soup.find_all('td')
    desc = soup.find_all('div', class_='res-text')
    top_line = soup.find('span', class_='midhead').find_next_sibling('br')

    idx = find_idx(tds)
    data = []
    description = []

    # Extracting description text
    for des in desc:
        description.append(des.get_text(strip=True))
    
    # Extracting Title
    date = top_line.next_sibling.strip()

    # Extracting Boat, Team, Times
    for i in range(0, len(idx), 1):
        if i < len(idx) -1:
            sequence = tds[idx[i]:idx[i+1]]
            boat = sequence[0].get_text(strip=True)
            # Now get all Teams and times
            for n in range(1, len(sequence), 2):
                team = sequence[n].get_text(strip=True)
                time = sequence[n + 1].get_text(strip=True)
                if team and time:
                    data.append((boat, team, time, date, description[0]))
        else:
            # Now the last Idx
            sequence = tds[idx[-1]:]
            boat = sequence[0].get_text(strip=True)
            for n in range(1, len(sequence), 2):
                team = sequence[n].get_text(strip=True)
                time = sequence[n + 1].get_text(strip=True)
                if team and time:
                    data.append((boat, team, time, date, description[0]))
    return data

result_frame = []
for content in html_pages:
    for i, contents in enumerate(content):
        result_frame.append(pd.DataFrame(results(soup = contents)))
        print(f'We are on step:{i} from {len(html_content)}')
        # if i == len(html_content):
    # for i, contents in enumerate(html_content):
        # result_frame.append(pd.DataFrame(results(soup = contents)))
        # print(f'We are on step:{i} from {len(html_content)}')
        # if i == len(html_content):
        

# start = pd.DataFrame(result_frame[0])
# for i in result_frame[1:]:
# res = pd.concat([start, pd.DataFrame(i)])

We are on step:0 from 0
We are on step:1 from 0
We are on step:2 from 0
We are on step:3 from 0
We are on step:4 from 0
We are on step:5 from 0
We are on step:6 from 0
We are on step:7 from 0
We are on step:8 from 0
We are on step:9 from 0
We are on step:10 from 0
We are on step:11 from 0
We are on step:12 from 0
We are on step:13 from 0
We are on step:14 from 0
We are on step:15 from 0
We are on step:16 from 0
We are on step:17 from 0
We are on step:18 from 0
We are on step:19 from 0
We are on step:20 from 0
We are on step:21 from 0
We are on step:22 from 0
We are on step:23 from 0
We are on step:24 from 0
We are on step:25 from 0
We are on step:26 from 0
We are on step:27 from 0
We are on step:28 from 0
We are on step:29 from 0
We are on step:30 from 0
We are on step:31 from 0
We are on step:32 from 0
We are on step:33 from 0
We are on step:34 from 0
We are on step:35 from 0
We are on step:36 from 0
We are on step:37 from 0
We are on step:0 from 0
We are on step:1 from 0
We are on st

In [260]:
start = pd.DataFrame(result_frame[0])
for i in result_frame[1:]:
    res = pd.concat([start, pd.DataFrame(i)])
# print(result_frame)

In [261]:
resu = pd.concat(result_frame)

In [262]:
string = 'April 29, 2023 - Philadelphia, PA - Schuylkill'

li = string.split()
string = ' '.join(li[4:])
string

'Philadelphia, PA - Schuylkill'

In [263]:
from datetime import datetime

In [264]:
resu['Date'] = resu[3].apply(lambda x: datetime.strptime(' '.join(x.split()[:3]), '%B %d, %Y'))
resu['Location'] = resu[3].apply(lambda x: ''.join(x.split()[4:]))

In [265]:
res = resu.rename(columns={0: 'Boat', 1: 'Team', 2: 'Time', 3:'Date_place', 4:'Description'}, inplace=True)

In [266]:
resu

,Boat,Team,Time,Date_place,Description,Date,Location
0,Varsity 8,Northeastern,5:37.1,"April 30, 2023 - Charles River basin",Distance:200 metersConditions:strong cross tai...,2023-04-30,CharlesRiverbasin
1,Varsity 8,Wisconsin,5:46.0,"April 30, 2023 - Charles River basin",Distance:200 metersConditions:strong cross tai...,2023-04-30,CharlesRiverbasin
2,2nd Varsity 8,Northeastern,5:46.6,"April 30, 2023 - Charles River basin",Distance:200 metersConditions:strong cross tai...,2023-04-30,CharlesRiverbasin
3,2nd Varsity 8,Wisconsin,5:49.7,"April 30, 2023 - Charles River basin",Distance:200 metersConditions:strong cross tai...,2023-04-30,CharlesRiverbasin
4,3rd Varsity 8,Northeastern 4V,5:55.6,"April 30, 2023 - Charles River basin",Distance:200 metersConditions:strong cross tai...,2023-04-30,CharlesRiverbasin
...,...,...,...,...,...,...,...
5,Frosh/Novice 8,Syracuse,6:15.7,"March 23, 2002 - Onondaga Lake","Distance:2000 meterConditions:27 degrees, 10-1...",2002-03-23,OnondagaLake
6,2F/4V 4+,Brown,6:38.9,"March 23, 2002 - Onondaga Lake","Distance:2000 meterConditions:27 degrees, 10-1...",2002-03-23,OnondagaLake
7,2F/4V 4+,Syracuse,7:04.2,"March 23, 2002 - Onondaga Lake","Distance:2000 meterConditions:27 degrees, 10-1...",2002-03-23,OnondagaLake
8,3V8,Brown,6:09.9,"March 23, 2002 - Onondaga Lake","Distance:2000 meterConditions:27 degrees, 10-1...",2002-03-23,OnondagaLake


In [267]:
resu.to_csv('race_results.csv')

In [268]:
# Now we go onto scraping the poll page
# Get links to all poll pages

In [269]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1200")

driver = webdriver.Chrome(options=options, executable_path=PATH)
driver.set_page_load_timeout(25)

In [270]:
driver.get('https://www.row2k.com/polls/index.cfm?cat=college&type=EARC%20HW%20Men')

In [271]:
def links_poll():
    hrefs = []
    date = []
    soup = BeautifulSoup(driver.page_source, "html.parser")
    ul_list = soup.find_all('ul', class_='body-small')
    for items in ul_list:
        links = items.find_all('a')
        for link in links:
            hrefs.append(link.get('href'))
            date.append(link.text)
            # date.append(items.get('text'))
    return list(zip(hrefs, date))

# hrefs, date = links_poll()
# Function to select all years and scrape all links for pages
links_all_polls = []
for i in range(0, 19):
    dropdown = driver.find_element(By.XPATH, '//*[@id="year_sel"]')

    # create a Select object
    select = Select(dropdown)

    # get all options of the dropdown
    options = select.options
    select.select_by_index(i)
    links_list = links_poll()
    links_all_polls.append(links_list)

In [272]:
# now we go to all poll pages and scrape the results
links_all_polls

[[('index.cfm?cat=college&ID=906&type=EARC HW Men', 'April 26, 2023'),
  ('index.cfm?cat=college&ID=899&type=EARC HW Men', 'April 19, 2023'),
  ('index.cfm?cat=college&ID=890&type=EARC HW Men', 'April 12, 2023')],
 [('index.cfm?cat=college&ID=859&type=EARC HW Men', 'May 6, 2022'),
  ('index.cfm?cat=college&ID=851&type=EARC HW Men', 'April 27, 2022'),
  ('index.cfm?cat=college&ID=840&type=EARC HW Men', 'April 20, 2022'),
  ('index.cfm?cat=college&ID=832&type=EARC HW Men', 'April 14, 2022')],
 [('index.cfm?cat=college&ID=777&type=EARC HW Men', 'May 14, 2019'),
  ('index.cfm?cat=college&ID=776&type=EARC HW Men', 'May 8, 2019'),
  ('index.cfm?cat=college&ID=767&type=EARC HW Men', 'May 1, 2019'),
  ('index.cfm?cat=college&ID=757&type=EARC HW Men', 'April 24, 2019')],
 [('index.cfm?cat=college&ID=716&type=EARC HW Men', 'May 6, 2018'),
  ('index.cfm?cat=college&ID=712&type=EARC HW Men', 'May 2, 2018'),
  ('index.cfm?cat=college&ID=704&type=EARC HW Men', 'April 25, 2018'),
  ('index.cfm?cat=co

In [273]:
driver.get(f'https://www.row2k.com/polls/index.cfm?cat=college&ID=906&type=EARC%20HW%20Men')
# driver.get(f'https://www.row2k.com/polls/{hrefs[0]}')

In [274]:
BOATS = ['1V', '2V', '3V', '4V', '5V']
def find_idx(ls):
    idx = []
    for i, item in enumerate(ls):
        if str(item).find('colspan') != -1:
            idx.append(i)
        else:
            continue
    return idx

def results_poll(soup, date):
    '''
    Find Results of page, given soup element of that page and create List of columns [boat, team, time]
    '''
    # Using Selenium webdriver and find_element
    boats = ['1V', '2V', '3V', '4V', '5V']
    tables = soup.find_all('table')
    data = []
    weird_formatted = []
    for i, table in enumerate(tables):
        rows = table.find_all('tr')[1:]
        # print(rows)
        table_data = []
        for row in rows:
            try:
                cells = row.find_all('td')
                standing = cells[0].text.strip()
                team = cells[1].text.strip()
                points = cells[2].text.strip()
                boat = boats[i]
                table_data.append([boat, standing, team, points])
            except IndexError:
                print(f'Weird Data entry: {date}')
                weird_formatted.append(date)
        data.append(table_data)
    return data

def concat_poll(data, date):
    '''
    Takes in one page, and the scraped results as given by result_poll returns a concatenated dataframe
    '''
    df_frames_poll = []
    for frames in data:
        df_frames_poll.append(pd.DataFrame(frames))
    df = pd.concat(df_frames_poll)
    df['Date'] = date
    df = df.rename(columns={0:'Boat', 1:'Ranking', 2:'Team', 3:'Score'})
    return df

# We append all dataframes to a List so that we can create the ultimate dataframe
all_frames_poll = []
for links in links_all_polls:
    for link in links:
        # Get page
        driver.get(f'https://www.row2k.com/polls/{link[0]}')
        result = results_poll(BeautifulSoup(driver.page_source, 'html.parser'), link[1])
        df = concat_poll(result, link[1])
        all_frames_poll.append(df)

Weird Data entry: April 18, 2017
Weird Data entry: April 18, 2017
Weird Data entry: April 18, 2017
Weird Data entry: April 18, 2017
Weird Data entry: May 9, 2014
Weird Data entry: May 3, 2013
Weird Data entry: May 3, 2013
Weird Data entry: May 3, 2013
Weird Data entry: April 24, 2009
Weird Data entry: April 24, 2009
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: April 20, 2004
Weird Data entry: Apri

In [275]:
df = pd.concat(all_frames_poll)
df

,Boat,Ranking,Team,Score,Date
0,1V,1,Yale University,1.00,"April 26, 2023"
1,1V,2,Princeton University,2.43,"April 26, 2023"
2,1V,3,Syracuse University,2.50,"April 26, 2023"
3,1V,4,Dartmouth College,3.64,"April 26, 2023"
4,1V,5,Harvard University,4.93,"April 26, 2023"
...,...,...,...,...,...
64,1V,3),NAV,2.5,"April 23, 2003"
65,1V,4),PRI,3.5,"April 23, 2003"
66,1V,5),NOR,4.5,"April 23, 2003"
67,1V,6),COR,6,"April 23, 2003"


In [276]:
df.to_csv('coches_poll.csv')

In [277]:
# Last thing we need to scrape are the EARC Championship results. That way we can see how the polls stack up against the actual
